In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from pandas import DataFrame
from dataclasses import dataclass
from lsst.daf.butler import Butler, DimensionRecord
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.simonyi.mountData import getAzElRotDataForExposure
from lsst.summit.utils.efdUtils import getEfdData
from lsst_efd_client import EfdClient
from astropy.coordinates import EarthLocation
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
expId = 2025052000245
dataId = {'exposure': expId, 'detector': 94, 'instrument': 'LSSTCam'}
expRecord = butlerUtils.getExpRecordFromDataId(butler, dataId)
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
butler.registry.queryDimensionRecords?

In [ ]:
expId = 2025052000321
instrument='LSSTCam'
where = "exposure.id=expId AND instrument='LSSTCam'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()
#print(expRecord)

In [ ]:
[azRate, elRate, rotRate] = getLinearRates(expRecord)

In [ ]:
rotRate

In [ ]:
mountData = getAzElRotDataForExposure(client, expRecord)

In [ ]:
azimuthData = mountData.azimuthData
azimuthData["linearModel"].plot(color='red', ls='--')
azimuthData["actualPosition"].plot(color='green')

In [ ]:
elevationData = mountData.elevationData
elevationData["linearModel"].plot(color='red', ls='--')
elevationData["actualPosition"].plot(color='green')

In [ ]:
rotationData = mountData.rotationData
rotationData["linearModel"].plot(color='red', ls='--')
rotationData["actualPosition"].plot(color='green')

In [ ]:
rotationData = mountData.rotationData
rotValues = np.asarray(rotationData["actualPosition"])
rotValTimes = np.asarray(rotationData["timestamp"])
rotModelValues = np.zeros_like(rotValues)
rotMedian = np.median(rotValues)
rotTimesMedian = np.median(rotValTimes)
rotModelValues = rotMedian + rotRate * (rotValTimes - rotTimesMedian)
rotationData["linearModel"] = rotModelValues
rotLinearError = (rotValues - rotModelValues) * 3600
rotLinearRms = np.sqrt(np.mean(rotLinearError * rotLinearError))


In [ ]:
plt.plot(rotValTimes, rotLinearError)

In [ ]:
azimuthData = mountData.azimuthData
azimuthData["azError"].plot(color='green')

In [ ]:
elevationData = mountData.elevationData
elevationData["elError"].plot(color='green')

In [ ]:
rotationData = mountData.rotationData
rotationData["rotError"].plot(color='green')

In [ ]:
[azRate, elRate, rotRate] = buildSimplePointing(expRecord)

In [ ]:
azRate

In [ ]:
azimuthData = mountData.azimuthData
azValues = np.asarray(azimuthData["actualPosition"])
azValTimes = np.asarray(azimuthData["actualPositionTimestamp"])
azModelValues = np.zeros_like(azValues)
azMedian = np.median(azValues)
azTimesMedian = np.median(azValTimes)
azModelValues = azMedian + azRate * (azValTimes - azTimesMedian)

In [ ]:
plt.plot(azValTimes, azValues, color='green')
plt.plot(azValTimes, azModelValues, color='red', ls='--')

In [ ]:
dT = (expRecord.timespan.end - expRecord.timespan.begin).value

In [ ]:
dT

In [ ]:
rot["actualPosition"].plot(color='blue', label='actualPosition')
rot["demandPosition"].plot(color='red', ls='--', label='demandPosition')
plt.legend()
plt.title(f"Rotator position {expId}")
plt.ylabel("Rotator angle (degrees)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Rotation_Data_{expId}.png")

In [ ]:
expId = 2025052000321
instrument='LSSTCam'
where = "exposure.id=expId AND instrument='LSSTCam'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()
#print(expRecord)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
fig = plotMountErrors(mountData, mountErrors)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/New_Mount_Plot_{expId}.png")

In [ ]:
@dataclass
class MountData:
    begin: Time
    end: Time
    azimuthData: DataFrame
    elevationData: DataFrame
    rotationData: DataFrame
    rotationTorques: DataFrame
    camhexData: DataFrame
    m2hexData: DataFrame
    includedPrePadding: float
    includedPostPadding: float
    expRecord: DimensionRecord | None


def getAzElRotHexDataForPeriod(
    client: EfdClient,
    begin: Time,
    end: Time,
    prePadding: float = 0,
    postPadding: float = 0,
    maxDeltaT: float = 1.0e-3,
) -> MountData:
    azimuthData = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    elevationData = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    rotationData = getEfdData(
        client,
        "lsst.sal.MTRotator.rotation",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    rotationTorques = getEfdData(
        client,
        "lsst.sal.MTRotator.motors",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    hexData = getEfdData(
        client,
        "lsst.sal.MTHexapod.application",
        begin=begin,
        end=end,
        prePadding=prePadding,
        postPadding=postPadding,
    )
    camhexData = hexData[hexData['salIndex'] == 1]
    m2hexData = hexData[hexData['salIndex'] == 2]

    def calcDeltaT(params, args):
        # This calculates the deltaT needed
        # to make the median(error) = 0
        [values, valTimes, demand, demTimes] = args
        [deltaT] = params
        demandInterp = np.interp(valTimes, demTimes + deltaT, demand)
        error = (values - demandInterp) * 3600
        value = abs(np.median(error))
        return value

    azValues = np.asarray(azimuthData["actualPosition"])
    azValTimes = np.asarray(azimuthData["actualPositionTimestamp"])
    azDemand = np.asarray(azimuthData["demandPosition"])
    azDemTimes = np.asarray(azimuthData["demandPositionTimestamp"])
    elValues = np.asarray(elevationData["actualPosition"])
    elValTimes = np.asarray(elevationData["actualPositionTimestamp"])
    elDemand = np.asarray(elevationData["demandPosition"])
    elDemTimes = np.asarray(elevationData["demandPositionTimestamp"])

    # Calculate the deltaT needed to drive the median(error) to zero
    args = [azValues, azValTimes, azDemand, azDemTimes]
    x0 = [0.0]
    result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
    deltaTAz = result.x[0]

    args = [elValues, elValTimes, elDemand, elDemTimes]
    x0 = [0.0]
    result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
    deltaTEl = result.x[0]

    azDemandInterp = np.interp(azValTimes, azDemTimes + deltaTAz, azDemand)
    elDemandInterp = np.interp(elValTimes, elDemTimes + deltaTEl, elDemand)

    azError = (azValues - azDemandInterp) * 3600
    elError = (elValues - elDemandInterp) * 3600

    rotValues = np.asarray(rotationData["actualPosition"])
    rotDemand = np.asarray(rotationData["demandPosition"])
    rotError = (rotValues - rotDemand) * 3600

    azimuthData["azError"] = azError
    elevationData["elError"] = elError
    rotationData["rotError"] = rotError



    mountData = MountData(
        begin, end, azimuthData, elevationData, rotationData, rotationTorques, camhexData,
        m2hexData, prePadding, postPadding, None
    )
    return mountData


def getAzElRotHexDataForExposure(
    client: EfdClient, expRecord: DimensionRecord, prePadding: float = 0, postPadding: float = 0
) -> MountData:

    begin = expRecord.timespan.begin
    end = expRecord.timespan.end
    mountData = getAzElRotHexDataForPeriod(client, begin, end, prePadding, postPadding)
    mountData.expRecord = expRecord
    return mountData

In [ ]:
expId = 2025052000321
instrument='LSSTCam'
where = "exposure.id=expId AND instrument='LSSTCam'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()


In [ ]:
mountData = getAzElRotHexDataForExposure(client, expRecord)

In [ ]:
mountData.camhexData

In [ ]:
NON_TRACKING_IMAGE_TYPES = ["BIAS", "FLAT"]

COMCAM_ANGLE_TO_EDGE_OF_FIELD_ARCSEC = 1800.0
LSSTCAM_ANGLE_TO_EDGE_OF_FIELD_ARCSEC = 8500.0

# These levels determine the colouring of the cells in the RubinTV.
# Yellow for warning level, red for bad level
MOUNT_IMAGE_WARNING_LEVEL = 0.05
MOUNT_IMAGE_BAD_LEVEL = 0.10  # and red for this

N_REPLACED_WARNING_LEVEL = 999999  # fill these values in once you've spoken to Craig and Brian
N_REPLACED_BAD_LEVEL = 999999  # fill these values in once you've spoken to Craig and Brian

SIMONYI_LOCATION = EarthLocation.of_site("Rubin:Simonyi")
EARTH_ROTATION = 15.04106858  # degrees/hour


@dataclass
class MountErrors:
    azRms: float
    elRms: float
    rotRms: float
    imageAzRms: float
    imageElRms: float
    imageRotRms: float
    imageImpactRms: float
    residualFiltering: bool
    nReplacedAz: int
    nReplacedEl: int


def tickFormatter(value: float, tick_number: float) -> str:
    # Convert the value to a string without subtracting large numbers
    # tick_number is unused.
    return f"{value:.2f}"


def calculateMountErrors(
    expRecord: DimensionRecord,
    client: EfdClient,
    maxDelta: float = 0.1,
    doFilterResiduals: bool = False,
    useMockPointingModelResidualsAboveAzEl: float = 10.0,
    useMockPointingModelResidualsAboveRot: float = 15.0,
) -> tuple[MountErrors, MountData] | tuple[None, None]:
    """Queries the EFD over a given exposure and calculates the RMS errors
    for the axes, optionally using a pointing model to calculate residuals.

    Parameters
    ----------
    expRecord : `DimensionRecord`
        The exposure record containing the necessary fields for calculations.
    client : `EfdClient`
        The EFD client to query for mount data.
    maxDelta : `float`, optional
        The maximum delta for filtering bad values, by default 0.1.
    doFilterResiduals : `bool`, optional
        Whether to filter residuals.
    useMockPointingModelResidualsAboveAzEl : `float`, optional
        The threshold above which to use the mock pointing model residuals, as
        an RMS, in arcseconds, for the azimuth and elevation axes.
    useMockPointingModelResidualsAboveRot : `float`, optional
        The threshold above which to use the mock pointing model residuals, as
        an RMS, in arcseconds, for the rotator.

    Returns
    -------
    tuple[MountErrors, MountData] | tuple[None, None]
        A tuple containing the mount errors and mount data, or (None, None) if
        the exposure type is non-tracking.
    """
    logger = logging.getLogger(__name__)

    imgType = expRecord.observation_type.upper()
    if imgType in NON_TRACKING_IMAGE_TYPES:
        logger.info(f"Skipping mount torques for non-tracking image type {imgType} for {expRecord.id}")
        return None, None

    mountData = getAzElRotHexDataForExposure(client, expRecord)

    elevation = 90 - expRecord.zenith_angle

    azError = mountData.azimuthData["azError"].to_numpy()
    elError = mountData.elevationData["elError"].to_numpy()
    rotError = mountData.rotationData["rotError"].to_numpy()
    nReplacedAz = 0
    nReplacedEl = 0
    if doFilterResiduals:
        # Filtering out bad values
        nReplacedAz = filterBadValues(azError, maxDelta)
        nReplacedEl = filterBadValues(elError, maxDelta)
        mountData.azimuthData["azError"] = azError
        mountData.elevationData["elError"] = elError

    # Calculate the linear demand model
    if len(mountData.azimuthData) == len(mountData.elevationData):
        azModelValues, elModelValues = getAltAzOverPeriod(expRecord, nPoints=len(mountData.azimuthData))
    else:
        azModelValues, _ = getAltAzOverPeriod(expRecord, nPoints=len(mountData.azimuthData))
        _, elModelValues = getAltAzOverPeriod(expRecord, nPoints=len(mountData.elevationData))

    _, _, rotRate = getLinearRates(expRecord)

    azimuthData = mountData.azimuthData
    azValues = np.asarray(azimuthData["actualPosition"])
    azMedian = np.median(azValues)
    azModelMedian = np.median(azModelValues)
    # subtract the overall offset
    azModelValues -= azModelMedian - azMedian
    azimuthData["linearModel"] = azModelValues
    azLinearError = (azValues - azModelValues) * 3600
    azLinearRms = np.sqrt(np.mean(azLinearError * azLinearError))
    if azLinearRms > useMockPointingModelResidualsAboveAzEl:
        logger.warning(
            f"Azimuth pointing model RMS error {azLinearRms:.3f} arcsec is above threshold of "
            f"{useMockPointingModelResidualsAboveAzEl:.3f} arcsec, calculating errors vs astropy."
        )
        # If linear error is large, replace demand errors with linear error
        azimuthData["azError"] = azLinearError

    elevationData = mountData.elevationData
    elValues = np.asarray(elevationData["actualPosition"])
    elMedian = np.median(elValues)
    elModelMedian = np.median(elModelValues)
    # subtract the overall offset
    elModelValues -= elModelMedian - elMedian
    elevationData["linearModel"] = elModelValues
    elLinearError = (elValues - elModelValues) * 3600
    elLinearRms = np.sqrt(np.mean(elLinearError * elLinearError))
    if elLinearRms > useMockPointingModelResidualsAboveAzEl:
        logger.warning(
            f"Elevation pointing model RMS error {elLinearRms:.3f} arcsec is above threshold of "
            f"{useMockPointingModelResidualsAboveAzEl:.3f} arcsec, calculating errors vs astropy."
        )
        # If linear error is large, replace demand errors with linear error
        elevationData["elError"] = elLinearError

    rotationData = mountData.rotationData
    rotValues = np.asarray(rotationData["actualPosition"])
    rotValTimes = np.asarray(rotationData["timestamp"])
    rotModelValues = np.zeros_like(rotValues)
    rotMedian = np.median(rotValues)
    rotTimesMedian = np.median(rotValTimes)
    rotModelValues = rotMedian + rotRate * (rotValTimes - rotTimesMedian)
    rotationData["linearModel"] = rotModelValues
    rotLinearError = (rotValues - rotModelValues) * 3600
    rotLinearRms = np.sqrt(np.mean(rotLinearError * rotLinearError))
    if rotLinearRms > useMockPointingModelResidualsAboveRot:
        logger.warning(
            f"Rotation pointing model RMS error {rotLinearRms:.3f} arcsec is above threshold of "
            f"{useMockPointingModelResidualsAboveAzEl:.3f} arcsec, calculating errors vs astropy."
        )
        # If linear error is large, replace demand errors with linear error
        rotationData["rotError"] = rotLinearError

    azError = mountData.azimuthData["azError"].to_numpy()
    elError = mountData.elevationData["elError"].to_numpy()
    rotError = mountData.rotationData["rotError"].to_numpy()

    azRms = np.sqrt(np.mean(azError * azError))
    elRms = np.sqrt(np.mean(elError * elError))
    rotRms = np.sqrt(np.mean(rotError * rotError))

    # Calculate Image impact RMS
    imageAzRms = azRms * np.cos(elevation * np.pi / 180.0)
    imageElRms = elRms
    imageRotRms = rotRms * COMCAM_ANGLE_TO_EDGE_OF_FIELD_ARCSEC * np.pi / 180.0 / 3600.0
    imageImpactRms = np.sqrt(imageAzRms**2 + imageElRms**2 + imageRotRms**2)

    mountErrors = MountErrors(
        azRms=float(azRms),
        elRms=float(elRms),
        rotRms=float(rotRms),
        imageAzRms=float(imageAzRms),
        imageElRms=float(imageElRms),
        imageRotRms=float(imageRotRms),
        imageImpactRms=float(imageImpactRms),
        residualFiltering=doFilterResiduals,
        nReplacedAz=nReplacedAz,
        nReplacedEl=nReplacedEl,
    )

    return (mountErrors, mountData)

def getAltAzOverPeriod(
    expRecord: DimensionRecord,
    nPoints: int,
) -> tuple[np.ndarray, np.ndarray]:
    """Get the AltAz coordinates over a period.

    Parameters
    ----------
    begin : `Time`
        The beginning of the period.
    end : `Time`
        The end of the period.
    target : `SkyCoord`
        The sky coordinates to track.
    nPoints : `int`, optional
        The number of points to sample, by default 100.

    Returns
    -------
    tuple[np.ndarray, np.ndarray]
        The azimuth and elevation coordinates in degrees.
    """
    begin = expRecord.timespan.begin
    end = expRecord.timespan.end
    times = begin + (end - begin) * np.linspace(0, 1, nPoints)
    target = SkyCoord(expRecord.tracking_ra * u.deg, expRecord.tracking_dec * u.deg)
    altAzFrame = AltAz(obstime=times, location=SIMONYI_LOCATION)
    targetAltAz = target.transform_to(altAzFrame)
    return targetAltAz.az.degree, targetAltAz.alt.degree

def getLinearRates(expRecord: DimensionRecord) -> tuple[float, float, float]:
    """Calculate the linear rates of motion for az, el, and rotation during an
    exposure.

    The rates are calculated based on the tracking RA and Dec, azimuth, zenith
    angle, and the exposure timespan. The rates are returned in degrees per
    second.

    Parameters
    ----------
    expRecord : `DimensionRecord`
        The exposure record containing the necessary fields for calculations.

    Returns
    -------
    azRate, elRate, rotRate: `tuple`[`float`, `float`, `float`]
        The azimuth rate, elevation rate, and rotator rate in degrees per
        second.
    """
    begin: Time = expRecord.timespan.begin
    end: Time = expRecord.timespan.end
    dT: float = (expRecord.timespan.end - expRecord.timespan.begin).value * 86400.0
    rotRate = (
        -EARTH_ROTATION
        * np.cos(SIMONYI_LOCATION.lat.rad)
        * np.cos(expRecord.azimuth * u.deg)
        / np.cos((90.0 - expRecord.zenith_angle) * u.deg)
        / 3600.0
    )
    skyLocation = SkyCoord(expRecord.tracking_ra * u.deg, expRecord.tracking_dec * u.deg)
    altAz1 = AltAz(obstime=begin, location=SIMONYI_LOCATION)
    altAz2 = AltAz(obstime=end, location=SIMONYI_LOCATION)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    elRate = float((obsAltAz2.alt.deg - obsAltAz1.alt.deg) / dT)
    azRate = float((obsAltAz2.az.deg - obsAltAz1.az.deg) / dT)

    # All rates are in degrees / second
    return azRate, elRate, float(rotRate.value)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
mountData.camhexData

In [ ]:
fig = plotMountErrors(mountData, mountErrors)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/New_Mount_Plot_{expId}.png")

In [ ]:
def plotMountErrors(
    mountData: MountData,
    mountErrors: MountErrors,
    figure=None,
    saveFilename: str = "",
) -> Figure:
    imageImpactRms = mountErrors.imageImpactRms
    expRecord = mountData.expRecord
    if expRecord is not None:
        dayObsString = dayObsIntToString(expRecord.day_obs)
        dataIdString = f"{expRecord.instrument} {dayObsString} - seqNum {expRecord.seq_num}"
        title = f"{dataIdString} - Exposure time = {expRecord.exposure_time:.1f}s"

    if figure is None:
        figure = plt.figure(figsize=(12, 8))
    else:
        figure.clear()
        ax = figure.gca()
        ax.clear()

    utc = ZoneInfo("UTC")
    chile_tz = ZoneInfo("America/Santiago")

    # Function to convert UTC to Chilean time
    def offset_time_aware(utc_time):
        # Ensure the time is timezone-aware in UTC
        if utc_time.tzinfo is None:
            utc_time = utc.localize(utc_time)
        return utc_time.astimezone(chile_tz)

    [[ax1, ax4], [ax2, ax5], [ax3, ax6]] = figure.subplots(
        3,
        2,
        sharex="col",
        sharey=False,
        gridspec_kw={"wspace": 0.25, "hspace": 0, "height_ratios": [2.5, 1, 1], "width_ratios": [1.5, 1]},
    )
    # [ax1, ax4] = [azimuth, rotator]
    # [ax2, ax5] = [azError, rotError]
    # [ax3, ax6] = [azTorque, rotTorque]

    # Use the native color cycle for the lines. Because they're on
    # different axes they don't cycle by themselves
    axs = [ax1, ax2, ax3, ax4, ax5, ax6]
    lineColors = [p["color"] for p in plt.rcParams["axes.prop_cycle"]]
    nColors = len(lineColors)
    colorCounter = 0

    ax1.plot(
        mountData.azimuthData["actualPosition"],
        label="Azimuth position",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax1.plot(
        mountData.azimuthData["linearModel"],
        label="Azimuth linear model",
        ls="--",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax1.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax1.set_ylabel("Azimuth (degrees)")

    ax1_twin = ax1.twinx()
    ax1_twin.plot(
        mountData.elevationData["actualPosition"],
        label="Elevation position",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax1_twin.plot(
        mountData.elevationData["linearModel"],
        label="Elevation linear model",
        ls="--",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1

    ax2.plot(
        mountData.azimuthData["azError"],
        label="Azimuth tracking error",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax2.plot(
        mountData.elevationData["elError"],
        label="Elevation tracking error",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax2.axhline(0.01, ls="-.", color="black")
    ax2.axhline(-0.01, ls="-.", color="black")
    ax2.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax2.set_ylabel("Tracking error (arcsec)")
    ax2.set_xticks([])  # remove x tick labels on the hidden upper x-axis
    ax2.set_ylim(-0.05, 0.05)
    ax2.set_yticks([-0.04, -0.02, 0.0, 0.02, 0.04])
    ax2.legend()
    ax2.text(0.1, 0.9, f"Image impact RMS = {imageImpactRms:.3f} arcsec (with rot).", transform=ax2.transAxes)
    if mountErrors.residualFiltering:
        ax2.text(
            0.1,
            0.8,
            (
                f"{mountErrors.nReplacedAz} bad az values and "
                f"{mountErrors.nReplacedEl} bad el values were replaced"
            ),
            transform=ax2.transAxes,
        )
    ax3_twin = ax3.twinx()
    ax3.plot(
        mountData.azimuthData["actualTorque"],
        label="Azimuth torque",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax3_twin.plot(
        mountData.elevationData["actualTorque"],
        label="Elevation torque",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax3.set_ylabel("Azimuth torque (Nm)")
    ax3_twin.set_ylabel("Elevation torque (Nm)")
    ax3.set_xlabel("Time (UTC)")  # yes, it really is UTC, matplotlib converts this automatically!

    # put the ticks at an angle, and right align with the tick marks
    ax3.set_xticks(ax3.get_xticks())  # needed to supress a user warning
    xlabels = ax3.get_xticks()
    ax3.set_xticklabels(xlabels)
    ax3.tick_params(axis="x", rotation=45)
    ax3.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

    ax4.plot(
        mountData.rotationData["actualPosition"],
        label="Rotator position",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax4.plot(
        mountData.rotationData["linearModel"],
        label="Rotator linearModel",
        ls="--",
        c=lineColors[colorCounter % nColors],
    )
    colorCounter += 1
    ax4.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax4.yaxis.tick_right()
    ax4.set_ylabel("Rotator angle (degrees)")
    ax4.yaxis.set_label_position("right")
    ax5.plot(
        mountData.rotationData["rotError"],
        c=lineColors[colorCounter % nColors],
    )

    colorCounter += 1
    ax5.axhline(0.1, ls="-.", color="black")
    ax5.axhline(-0.1, ls="-.", color="black")
    ax5.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax5.set_ylabel("Tracking error (arcsec)")
    ax5.tick_params(labelbottom=False)  # Hide x-axis tick labels without removing ticks
    ax5.set_ylim(-0.5, 0.5)
    ax5.set_yticks([-0.4, -0.2, 0.0, 0.2, 0.4])
    ax5.yaxis.tick_right()
    ax5.yaxis.set_label_position("right")

    ax6.plot(mountData.rotationTorques["torque0"], label="Torque0", c=lineColors[colorCounter % nColors])
    colorCounter += 1
    ax6.plot(mountData.rotationTorques["torque1"], label="Torque1", c=lineColors[colorCounter % nColors])
    ax6.set_ylabel("Rotator torque (Nm)")
    ax6.set_xlabel("Time (UTC)")  # yes, it really is UTC, matplotlib converts this automatically!
    # put the ticks at an angle, and right align with the tick marks
    ax6.set_xticks(ax6.get_xticks())  # needed to supress a user warning
    xlabels = ax6.get_xticks()
    ax6.set_xticklabels(xlabels)
    ax6.tick_params(axis="x", rotation=45)
    ax6.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax6.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax6.yaxis.tick_right()
    ax6.yaxis.set_label_position("right")
    ax6.legend()

    ax1_twin.yaxis.set_major_formatter(FuncFormatter(tickFormatter))
    ax1_twin.set_ylabel("Elevation (degrees)")
    ax1.tick_params(labelbottom=False)  # Hide x-axis tick labels without removing ticks
    # combine the legends and put inside the plot
    handles1a, labels1a = ax1.get_legend_handles_labels()
    handles1b, labels1b = ax1_twin.get_legend_handles_labels()
    handles2a, labels2a = ax3.get_legend_handles_labels()
    handles2b, labels2b = ax3_twin.get_legend_handles_labels()
    handles = handles1a + handles1b + handles2a + handles2b
    labels = labels1a + labels1b + labels2a + labels2b
    # ax2 is "in front" of ax1 because it has the vlines plotted on it, and
    # vlines are on ax2 so that they appear at the bottom of the legend, so
    # make sure to plot the legend on ax2, otherwise the vlines will go on
    # top of the otherwise-opaque legend.
    ax1_twin.legend(handles, labels, facecolor="white", framealpha=1)

    ax1.set_title("Azimuth and Elevation")
    ax4.set_title("Rotator")
    ax4.legend()
    figure.subplots_adjust(top=0.85)  # Adjust the top margin to make room for the suptitle
    figure.suptitle(title, fontsize=14, y=1.04)  # Adjust y to move the title up

    # Create the upper axis for Chilean time
    ax1_twiny = ax1.twiny()
    ax1_twiny.set_xlim(ax1.get_xlim())  # Set the limits of the upper axis to match the lower axis
    utcTicks = ax1.get_xticks()  # Use the same ticks as the lower UTC axis
    utcTickLabels = [num2date(tick, tz=utc) for tick in utcTicks]
    chileTickLabels = [offset_time_aware(label) for label in utcTickLabels]
    # Set the same tick positions but with Chilean time labels
    ax1_twiny.set_xticks(utcTicks)
    ax1_twiny.set_xticklabels([tick.strftime("%H:%M:%S") for tick in chileTickLabels])
    ax1_twiny.tick_params(axis="x", rotation=45)
    ax1_twiny.set_xlabel("Time (Chilean Time)")

    ax4_twiny = ax4.twiny()
    ax4_twiny.set_xlim(ax4.get_xlim())  # Set the limits of the upper axis to match the lower axis
    utcTicks = ax4.get_xticks()  # Use the same ticks as the lower UTC axis
    utcTickLabels = [num2date(tick, tz=utc) for tick in utcTicks]
    chileTickLabels = [offset_time_aware(label) for label in utcTickLabels]
    # Set the same tick positions but with Chilean time labels
    ax4_twiny.set_xticks(utcTicks)
    ax4_twiny.set_xticklabels([tick.strftime("%H:%M:%S") for tick in chileTickLabels])
    ax4_twiny.tick_params(axis="x", rotation=45)
    ax4_twiny.set_xlabel("Time (Chilean Time)")

    # Add exposure start and end:
    for ax in axs:
        if expRecord is not None:
            # assert expRecord is not None, "expRecord is None"
            ax.axvline(expRecord.timespan.begin.utc.datetime, ls="--", color="green")
            ax.axvline(expRecord.timespan.end.utc.datetime, ls="--", color="red")

    if saveFilename:
        figure.savefig(saveFilename, bbox_inches="tight")

    return figure


